In [25]:
import tensorflow as tf
import tensorboard as tb

import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Ridge

In [2]:
x = pd.read_csv('c:/Users/qqmai/Desktop/mai.zjsh/sku_pred/luo.sz/appendata/y2018_feature_上海_春.csv',
header=0,engine='python')

In [3]:
is_select = [*map(lambda x:all(['huan' not in x,'mean' not in x,'Unnamed' not in x,
'bin' not in x,'first' not in x,'last' not in x,'weight' not in x,'cumsum' not in x,
'diff' not in x,'rank' not in x,'ratio' not in x])
,x.columns)]
x = x[x.columns[is_select]]

In [4]:
x['categ'] = x['product_code'].apply(lambda x:x[-2])
x['season'] = x['product_code'].apply(lambda x:x[-3])
x['color'] = x['product_code'].apply(lambda x:x[-5])

In [5]:
x.head()

,product_code,date_week_four,qty_sum_one,qty_sum_two,qty_sum_three,qty_sum_four,discount_one,discount_two,discount_three,discount_four,...,ftr+store4,ftr+store5,ftr_qty1,ftr_qty2,ftr_qty3,ftr_qty4,ftr_qty5,categ,season,color
0,B4CS2K1DDA1AM8,3.0,3.0,4.0,4.0,5.0,0.740452,0.722191,0.763626,0.700111,...,58.0,58.000000,0.0,8.0,5.0,4.0,4.0,M,A,A
1,B4CS2K1DDU1AM8,3.0,5.0,2.0,9.0,28.0,0.741046,0.721913,0.730070,0.738320,...,58.0,54.142857,14.0,15.0,16.0,15.0,12.0,M,A,U
2,B4DS2J1DDU1AM8,3.0,13.0,5.0,8.0,7.0,0.709072,0.725130,0.706986,0.701326,...,57.0,58.714286,1.0,1.0,11.0,11.0,17.0,M,A,U
3,B4PS2X1DDS1AQ8,3.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.690338,0.690338,...,43.0,43.000000,0.0,0.0,2.0,1.0,2.0,Q,A,S
4,B4PS2X1DDU1AQ8,3.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.767171,0.767171,...,43.0,43.000000,0.0,0.0,1.0,2.0,7.0,Q,A,U


In [6]:
ohe = OneHotEncoder()
df_color = pd.DataFrame(ohe.fit_transform(x[['color']]).toarray())
df_color.columns = ['color_'+str(k) for k in df_color.columns]

df_categ = pd.DataFrame(ohe.fit_transform(x[['categ']]).toarray())
df_categ.columns = ['categ_'+str(k) for k in df_categ.columns]

df_season = pd.DataFrame(ohe.fit_transform(x[['season']]).toarray())
df_season.columns = ['season_'+str(k) for k in df_season.columns]

x1,y1 = x.iloc[:,2:-14],x.iloc[:,-8:-3].sum(axis=1) 
x1 = pd.concat([x1,df_color,df_categ,df_season],axis=1)
y1 = np.expand_dims(y1,axis=1)

In [7]:
X_train,X_test,y_train,y_test = train_test_split(x1,y1,test_size=0.3)

In [8]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((8194, 42), (3512, 42), (8194, 1), (3512, 1))

In [9]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

y_train = y_train.astype('float')
y_test = y_test.astype('float')

In [10]:
# input_field
input_field = [0]*24+[1]*14+[2]*2+[3]*2

In [11]:
# fea_index = tf.placeholder(tf.int32,shape=[None,None],name='feature_index')
# fea_value = tf.placeholder(tf.int32,shape=[None,None],name='feature_value')
# fea_label = tf.placeholder(tf.int16,shape=[None,None],name='feature_label')

# feature_size,embedding_size = 40,20

In [12]:
# #embeddings
# weights = dict()

# weights['feature_embeddings'] = tf.Variable(tf.random_normal([feature_size,embedding_size],0.0,0.01),
#     name='feature_embeddings')
# weights['feature_bias'] = tf.Variable(tf.random_normal([feature_size,1],0.0,1.0),
#     name='feature_bias')

In [13]:
input_size,field_size,vector_dim = 42,4,1

weights0 = tf.Variable(tf.truncated_normal([1]))
weights1 = tf.Variable(tf.truncated_normal([input_size]))

## input_x_size 表示 one hot code后的变量树
## field_size 表示 域数
## vector_dim 表示 每个向量v的长度
weights2 = tf.Variable(tf.truncated_normal([input_size,field_size,vector_dim]))



In [14]:
def inference(input_x,input_field,weights0,weights1,weights2):
    """计算回归模型输出的值"""

    secondValue = tf.reduce_sum(tf.multiply(weights1,input_x,name='secondValue')) ### 一次项的值

    firstTwoValue = tf.add(weights0, secondValue, name="firstTwoValue")

    thirdValue = tf.Variable(0.01,dtype=tf.float32)
    input_shape = input_size

    # for i in range(input_shape):
    #     fieldIndex1 = int(input_field[i])

    #     for j in range(i+1,input_shape):
    #         fieldIndex2 = int(input_field[j])
    #         vectorLeft = tf.convert_to_tensor([[i,j,k] for k in range(vector_dim)])
    #         weightLeft = tf.gather_nd(thirdWeight,vectorLeft)
    #         weightLeftAfterCut = tf.squeeze(weightLeft)

    #         vectorRight = tf.convert_to_tensor([[i,j,k] for k in range(vector_dim)])
    #         weightRight = tf.gather_nd(thirdWeight,vectorRight)
    #         weightRightAfterCut = tf.squeeze(weightRight)

    #         tempValue = tf.reduce_sum(tf.multiply(weightLeftAfterCut,weightRightAfterCut))

    #         indices2 = [i]
    #         indices3 = [j]

    #         xi = tf.squeeze(tf.gather_nd(input_x, indices2))
    #         xj = tf.squeeze(tf.gather_nd(input_x, indices3))

    #         product = tf.reduce_sum(tf.multiply(xi, xj))

    #         secondItemVal = tf.multiply(tempValue, product)

    #         tf.assign(thirdValue, tf.add(thirdValue, secondItemVal))

    # return tf.add(firstTwoValue,thirdValue)
    return firstTwoValue

In [22]:
lambda_w = tf.constant(10., name='lambda_w')
lambda_v = tf.constant(0.001, name='lambda_v')

x_input = tf.placeholder(tf.float32,[None,42])
y_input = tf.placeholder(tf.float32,[None,1])

y_ = inference(x_input,input_field,weights0,weights1,weights2)

# l2_norm = tf.reduce_sum(
#     tf.add(
#         tf.multiply(lambda_w, tf.pow(oneDimWeights, 2)),
#         tf.reduce_sum(tf.multiply(lambda_v, tf.pow(thirdWeight, 2)),axis=[1,2])
#     )
# )

l2_norm = tf.multiply(lambda_w,tf.reduce_sum(tf.pow(weights1,2)))

loss = tf.reduce_mean(tf.pow(tf.subtract(y_input,y_),2))
loss_ = tf.add(loss,l2_norm)

# train_step = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss_)
train_step = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss_)

tf.summary.scalar('loss',loss_)
summ_op = tf.summary.merge_all()

In [23]:
# tf.assign_add?
# tf.assign_add(loss,l2_norm)
type(summ_op)

tensorflow.python.framework.ops.Tensor

In [21]:
train_log_path = "./tmp/train_tmp"
test_log_path = "./tmp/test_tmp"


init = tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init)

    summary_writer_tr = tf.summary.FileWriter(train_log_path, graph=sess.graph)
    summary_writer_ts = tf.summary.FileWriter(test_log_path)

    for i in range(1000):
        # sess.run([train_step],feed_dict={x_input:x1.values, y_input:y1})

        _,lss_tr,w1,smm = sess.run([train_step,loss_,weights1,summ_op],
                                    feed_dict={x_input:X_train.values, y_input:y_train})

        if i%10 == 0:
            print(i)
            summary_writer_tr.add_summary(smm,i)

            lss_ts,w1,smm2 = sess.run([loss_,weights1,summ_op],
                                    feed_dict={x_input:X_test.values, y_input:y_test})
            summary_writer_ts.add_summary(smm2,i)

            print('train loss in round %d : %.2d' %(i,round(lss_tr,2)))
            print('test loss in round %d : %.2d' %(i,round(lss_ts,2)))

    # input_x_batch = x1[1]
    # input_y_batch = y1[1]
        # print(sess.run(train_step))

summary_writer.close()
sess.close()

0
train loss in round 0 : 1958782464
test loss in round 0 : 379873472
10
train loss in round 10 : 1761656448
test loss in round 10 : 342726112
20
train loss in round 20 : 1576742400
test loss in round 20 : 307858208
30
train loss in round 30 : 1405027840
test loss in round 30 : 275437312
40
train loss in round 40 : 1246964352
test loss in round 40 : 245549840
50
train loss in round 50 : 1102409856
test loss in round 50 : 218163136
60
train loss in round 60 : 970919296
test loss in round 60 : 193187376
70
train loss in round 70 : 851851456
test loss in round 70 : 170514944
80
train loss in round 80 : 744483904
test loss in round 80 : 150010304
90
train loss in round 90 : 648052416
test loss in round 90 : 131539224
100
train loss in round 100 : 561823360
test loss in round 100 : 114954912
110
train loss in round 110 : 485037024
test loss in round 110 : 100135248
120
train loss in round 120 : 416979584
test loss in round 120 : 86937232
130
train loss in round 130 : 356897376
test loss in 

In [28]:
w1.mean()

0.031287394

In [30]:
reg = Ridge(alpha=10)
reg.fit(X_train,y_train)
reg.coef_.mean()

0.5339930270038027

In [20]:
tf.__version__

'1.8.0'

In [21]:
os.getcwd()

'c:\\Users\\qqmai\\Desktop\\study\\study\\deep learning'